In [1]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import math
import os

use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  device: torch.device = torch.device("cpu"),


In [2]:
data_path = "nturgb+d_skeletons/"
broken_files_path = "NTU_RGBD120_samples_with_missing_skeletons.txt"

In [3]:
# training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = sorted([8, 10, 22, 23, 27, 21, 55, 2, 7]) #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 

# max_body_true = 1
# max_body_kinect = 1

num_joint = 25
max_frame = 300

In [4]:
class Skeleton_Dataset(Dataset):
    def __init__(self, data_path, broken_files_path=None, training_classes=None,
                 num_joint = 25, max_frame = 300, transform=None):
        
        
        def read_data(data_path, broken_files_path):
            labels = []
            files = []
            action_classes = {}
            counter = 0
            files_counter = {}
            with open(broken_files_path, 'r') as f:
                broken_files = f.read().split("\n")

            raw_files = os.listdir(data_path)
            num_frames = 0

            for filename in raw_files:
                if filename not in broken_files:
                    action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                    subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                    camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                    if action_class in training_classes and camera_id in training_cameras: 
                        if action_class in action_classes:
                            if files_counter[action_class] < 120:
                                files.append([filename,action_classes[action_class]])
                                files_counter[action_class] = files_counter[action_class] + 1
                        else:
                            action_classes.update({action_class : counter})
                            files_counter.update({action_class : 1})
                            counter+=1
                            files.append([filename,action_classes[action_class]])
            print("action classes: ", action_classes)
            print("action files: ", files_counter)

            return files, action_classes
        
        
        def read_skeleton_filter(file):
            with open(file, 'r') as f:
                skeleton_sequence = {}
                skeleton_sequence['numFrame'] = int(f.readline())
                skeleton_sequence['frameInfo'] = []
                for t in range(skeleton_sequence['numFrame']):
                    frame_info = {}
                    frame_info['numBody'] = int(f.readline())
                    frame_info['bodyInfo'] = []

                    for m in range(frame_info['numBody']):
                        body_info = {}
                        body_info_key = [
                            'bodyID', 'clipedEdges', 'handLeftConfidence',
                            'handLeftState', 'handRightConfidence', 'handRightState',
                            'isResticted', 'leanX', 'leanY', 'trackingState'
                        ]
                        body_info = {
                            k: float(v)
                            for k, v in zip(body_info_key, f.readline().split())
                        }
                        body_info['numJoint'] = int(f.readline())
                        body_info['jointInfo'] = []
                        for v in range(body_info['numJoint']):
                            joint_info_key = [
                                'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                                'orientationW', 'orientationX', 'orientationY',
                                'orientationZ', 'trackingState'
                            ]
                            joint_info = {
                                k: float(v)
                                for k, v in zip(joint_info_key, f.readline().split())
                            }
                            body_info['jointInfo'].append(joint_info)
                        frame_info['bodyInfo'].append(body_info)
                    skeleton_sequence['frameInfo'].append(frame_info)

            return skeleton_sequence

        def read_xyz(file, max_body=1, num_joint=25):
            seq_info = read_skeleton_filter(file)
            data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
            for n, f in enumerate(seq_info['frameInfo']):
                for m, b in enumerate(f['bodyInfo']):
                    for j, v in enumerate(b['jointInfo']):
                        if m < max_body and j < num_joint:
                            data[m, n, j, :] = [v['x'], v['y'], v['z']]

                        else:
                            pass

            return data
        
        
        def create_coords_blocks(test_file, chonk_len = 45):   
            frame_counter = 0
            new_labels = []
            new_frames = []
            blocks = []

            test_frames = read_xyz(data_path + test_file[0])[0]
            label = test_file[1]
            slice_len = chonk_len * int(len(test_frames)/chonk_len)


            for index in range(len(test_frames[:slice_len])):
                frame_counter += 1
                new_frames.append(test_frames[index].flatten())
                if frame_counter == chonk_len:
                    frame_counter = 0
                    blocks.append(np.array(new_frames))
                    new_labels = new_labels + [label]
                    new_frames = []


            return blocks, new_labels
        
        
        ##### список файлов с лейблами на каждый файл 
        working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
        
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(action_classes))}  #####
        ##################################################################
        for file in working_files_with_labels:
            frames_blocks, label = create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        data = pd.DataFrame(data_sq)
        labels = pd.DataFrame(labels_np)
        data['labels'] = labels
        

        self.data = data
        self.labels = data['labels'].astype('float32')
        self.transform = transform
        
           
    def __len__(self):
         return len(self.data)
        
        
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
        return (item, label)

In [5]:
dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = 25, 
                           max_frame = 300, transform=None)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\2051480725.py", line 1, in <cell line: 1>
    dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path,
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\2872100655.py", line 118, in __init__
    working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\2872100655.py", line 12, in read_data
    with open(broken_files_path, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'NTU_RGBD120_samples_with_missing_skeletons.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python

In [6]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),
                                                                      len(dataset) - int(0.75*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3620045888.py", line 1, in <cell line: 1>
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),
NameError: name 'dataset' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\P

In [7]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [8]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=9, bias=True)
)

In [9]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [10]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(250):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3086500835.py", line 11, in <cell line: 8>
    for i, data in enumerate(train_loader, 0):
NameError: name 'train_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\cor

In [11]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3599532051.py", line 7, in <cell line: 6>
    for i, data in enumerate(test_loader, 0):
NameError: name 'test_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\u

In [12]:
n_hidden = 128*3
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 384, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=384, out_features=9, bias=True)
)

In [13]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [14]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(250):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3086500835.py", line 11, in <cell line: 8>
    for i, data in enumerate(train_loader, 0):
NameError: name 'train_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\cor

In [15]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3599532051.py", line 7, in <cell line: 6>
    for i, data in enumerate(test_loader, 0):
NameError: name 'test_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\u

In [16]:
training_classes = sorted([8, 10, 22, 23, 27, 21, 55, 2, 7]) #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 


num_joint = 25
max_frame = 200

In [17]:
dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = num_joint, 
                           max_frame = max_frame, transform=None)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\610034384.py", line 1, in <cell line: 1>
    dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path,
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\2872100655.py", line 118, in __init__
    working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\2872100655.py", line 12, in read_data
    with open(broken_files_path, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'NTU_RGBD120_samples_with_missing_skeletons.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\

In [18]:
n_hidden = 128*3
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 384, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=384, out_features=9, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(250):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3086500835.py", line 11, in <cell line: 8>
    for i, data in enumerate(train_loader, 0):
NameError: name 'train_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\cor

In [20]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\azhur\AppData\Local\Temp\ipykernel_5596\3599532051.py", line 7, in <cell line: 6>
    for i, data in enumerate(test_loader, 0):
NameError: name 'test_loader' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\u